In [26]:
#Import all the libraries

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import Perceptron

from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [2]:
def load_dataset(path):
  """
  Loads a dataset from a given path.

  Args:
    path: The path to the dataset.

  Returns:
    The loaded dataset.
  """

  # Check if the path exists.
  if not os.path.exists(path):
    raise ValueError("The provided path does not exist.")

  # Load the dataset using the appropriate library based on the file format.
  # For example, you could use pandas to load a CSV file.
  dataset = pd.read_csv(path)

  return dataset


In [16]:
def preprocess_data(data):
    # Replace infinity or large values with NaN
    numeric_cols = data.select_dtypes(include=[np.number]).columns
    data[numeric_cols].replace([np.inf, -np.inf], np.nan, inplace=True)
    data[(data[numeric_cols].abs() > 1e10)] = np.nan

    # Drop one variable features
    one_variable_list = []
    for i in data.columns:
        if data[i].value_counts().nunique() < 2:
            one_variable_list.append(i)
    data.drop(one_variable_list, axis=1, inplace=True)
    data.columns =  data.columns.str.strip()

    # Dropping duplicate rows
    data = data.drop_duplicates(keep="first")
    data.reset_index(drop=True, inplace=True)

    # Dropping Null values
    data.dropna(inplace=True)

    return data

In [17]:
def correlation(data):
    # Exclude non-numeric columns
    numeric_data = data.select_dtypes(include=[np.number])

    # Calculate correlation matrix
    corr_matrix = numeric_data.corr().abs()

    # Create upper triangle matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

    # Find features with correlation greater than 0.95 and drop them
    to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
    data = data.drop(to_drop, axis=1)
    shape = data.shape
    
    return shape

In [18]:
def split_data(data):
    # data = data.replace([np.inf , -np.inf],inplace=True)
    X = data.drop(["Label"],axis=1)
    y = data["Label"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    # # Standardize the features
    # scaler = StandardScaler()
    # X_train = scaler.fit_transform(X_train)
    # X_test = scaler.transform(X_test)
    
    return X_train, X_test, y_train, y_test

In [19]:
def model_train(X_train, X_test, y_train, y_test):
    models = {
    # 'Naive Bayes': BernoulliNB(),
    'Decision Tree': DecisionTreeClassifier(),
    'KNN': KNeighborsClassifier(),
    'Quadratic Discriminant Analysis': QuadraticDiscriminantAnalysis(),
    'Perceptron': Perceptron(),
    'Random Forest': RandomForestClassifier()
}
    results = {}
    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred)
        results[name] = {'accuracy': accuracy, 'report': report}
    # Print the results
    print('Results:')
    print('Model | Accuracy | Report')
    print('---|---|---')
    for name, result in results.items():
        print(f'{name} | {result["accuracy"]}')
        print(f'{result["report"]}')

In [20]:
data = load_dataset("1.csv")

In [21]:
print(preprocess_data(data))

/tmp/ipykernel_192/391613588.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[numeric_cols].replace([np.inf, -np.inf], np.nan, inplace=True)


        Destination Port  Flow Duration  Total Fwd Packets  \
0                     80          38308                  1   
1                    389            479                 11   
2                     88           1095                 10   
3                    389          15206                 17   
4                     88           1092                  9   
...                  ...            ...                ...   
610789                53          32215                  4   
610790                53            324                  2   
610791             58030             82                  2   
610792                53        1048635                  6   
610793                53          94939                  4   

        Total Backward Packets  Total Length of Fwd Packets  \
0                            1                            6   
1                            5                          172   
2                            6                         3150   
3  

/tmp/ipykernel_192/391613588.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(inplace=True)


In [22]:
print(correlation(data))

(692703, 40)


In [ ]:
# Split data into X and y
X = data.drop("Label", axis=1)
y = data["Label"]

In [23]:
X_train, X_test, y_train, y_test = split_data(data)

In [29]:
def model_train(X_train, X_test, y_train, y_test):
    models = {
        'Naive Bayes': BernoulliNB(),
        'Decision Tree': DecisionTreeClassifier(),
        'KNN': KNeighborsClassifier(),
        'Quadratic Discriminant Analysis': QuadraticDiscriminantAnalysis(),
        'Perceptron': Perceptron(),
        'Random Forest': RandomForestClassifier()
    }
    
    results = {}
    for name, model in models.items():
        pipeline = Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),  # Impute missing values with mean
            ('model', model)
        ])
        
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        results[name] = accuracy
        print(f"{name} Accuracy: {accuracy}")
    
    return results

In [30]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
print(model_train(X_train, X_test, y_train, y_test))

Naive Bayes Accuracy: 0.6851762294194499
Decision Tree Accuracy: 0.9994658620913665
KNN Accuracy: 0.9950483972253701


/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


Quadratic Discriminant Analysis Accuracy: 0.9940595202864134
Perceptron Accuracy: 0.9054720263315553
Random Forest Accuracy: 0.9994514259316737
{'Naive Bayes': 0.6851762294194499, 'Decision Tree': 0.9994658620913665, 'KNN': 0.9950483972253701, 'Quadratic Discriminant Analysis': 0.9940595202864134, 'Perceptron': 0.9054720263315553, 'Random Forest': 0.9994514259316737}
